# Transfer Learning

## Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

## Carregando dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


## ResNet50



### Carregando o dataset , normalizando e redimensionando as imagens

In [ ]:
print("Carregando dataset CIFAR-10...")
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertendo para float32 e normalizando
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# ResNet50 normalmente espera imagens 224x224
# Vamos redimensionar para 64x64 para manter um bom equilíbrio entre performance e memória
print("Redimensionando imagens...")
x_train = tf.image.resize(x_train, (64, 64))
x_test = tf.image.resize(x_test, (64, 64))

# Pré-processamento específico do ResNet50
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Convertendo labels para one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print('Formato dos dados de treino:', x_train.shape)
print('Formato dos dados de teste:', x_test.shape)


Carregando dataset CIFAR-10...
Redimensionando imagens...
Formato dos dados de treino: (50000, 64, 64, 3)
Formato dos dados de teste: (10000, 64, 64, 3)


### Carregando modelo ResNet50

In [ ]:
print("\nCarregando modelo ResNet50...")
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(64, 64, 3)
)


Carregando modelo ResNet50...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


### Inserir novas camadas e criação do modelo final

Uso do `GlobalAveragePooling2D` calcula a média de cada canal de uma imagem, reduzindo sua dimensão para um vetor de características. Isso torna a rede neural mais leve, robusta e interpretável, sendo uma alternativa eficiente à camada Flatten.

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

# Adicionando novas camadas para classificação
# Usando GlobalAveragePooling2D em vez de Flatten para reduzir parâmetros
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Criando o modelo final
model = Model(inputs=base_model.input, outputs=predictions)


### Função de otimização e métricas

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nEstrutura do modelo:")
model.summary()


Estrutura do modelo:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 64, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 70, 70, 3)      │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 32, 32, 64)     │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 32, 32, 64)     │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 32, 32, 64)     │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 34, 34, 64)     │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 16, 16, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 16, 16, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 16, 16, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 16, 16, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 16, 16, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 16, 16, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 16, 16, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 16, 16, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 16, 16, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 24,770,698 (94.49 MB)

 Trainable params: 1,182,986 (4.51 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

### Data Augmentation

Utiliza o `ImageDataGenerator` do Keras para criar aumento de dados em tempo real, ou seja, ele gera novas imagens a partir das suas imagens originais durante o treinamento. Isso ajuda a prevenir overfitting, melhorar a generalização do modelo e aumentar o tamanho do seu conjunto de dados.

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

### Treinando o modelo

In [ ]:
epochs = 25
batch_size = 32

print("\nIniciando treinamento...")
# Treinando o modelo
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    verbose=1
)


Iniciando treinamento...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 107s 60ms/step - accuracy: 0.5389 - loss: 1.5758 - val_accuracy: 0.7450 - val_loss: 0.7878
Epoch 2/25
   1/1562 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.5625 - loss: 1.3587

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5625 - loss: 1.3587 - val_accuracy: 0.7439 - val_loss: 0.7886
Epoch 3/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 117s 51ms/step - accuracy: 0.6792 - loss: 0.9576 - val_accuracy: 0.7616 - val_loss: 0.7326
Epoch 4/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8125 - loss: 0.6130 - val_accuracy: 0.7617 - val_loss: 0.7340
Epoch 5/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 79s 50ms/step - accuracy: 0.7013 - loss: 0.9080 - val_accuracy: 0.7594 - val_loss: 0.7132
Epoch 6/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6875 - loss: 0.7633 - val_accuracy: 0.7595 - val_loss: 0.7117
Epoch 7/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 77s 49ms/step - accuracy: 0.7127 - loss: 0.8861 - val_accuracy: 0.7688 - val_loss: 0.6862
Epoch 8/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7188 - loss: 1.0106 - val_accuracy: 0.7697 - val_loss: 0.6851
Epoch 9/25
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.7142 - loss: 0.8757 - va

### Avaliação do modelo

In [ ]:
print("\nAvaliando o modelo...")
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Acurácia nos dados de teste: {test_accuracy:.4f}')
print(f'Loss nos dados de teste: {test_loss:.4f}')


Avaliando o modelo...
Acurácia nos dados de teste: 0.7895
Loss nos dados de teste: 0.6489


## VGG16

### Carregando o dataset

In [ ]:
print("Carregando dataset CIFAR-10...")
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Carregando dataset CIFAR-10...


### Redimensionando as imagens e divisão entre test e train

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Redimensionando as imagens para 48x48
print("Redimensionando imagens...")
x_train = tf.image.resize(x_train, (48, 48))
x_test = tf.image.resize(x_test, (48, 48))

# Pré-processamento específico do VGG16
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# Convertendo labels para one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print('Formato dos dados de treino:', x_train.shape)
print('Formato dos dados de teste:', x_test.shape)

Redimensionando imagens...
Formato dos dados de treino: (50000, 48, 48, 3)
Formato dos dados de teste: (10000, 48, 48, 3)


### Carregando o modelo

In [ ]:
print("\nCarregando modelo VGG16...")
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(48, 48, 3)
)


Carregando modelo VGG16...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


### Adição de novas camadas

In [ ]:
# Congelando as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Adicionando novas camadas para classificação
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

### Função de otimização e métricas

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

# Compilando o modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nEstrutura do modelo:")
model.summary()


Estrutura do modelo:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 48, 48, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 48, 48, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 48, 48, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 24, 24, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 12, 12, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 6, 6, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 3, 3, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 15,111,242 (57.64 MB)

 Trainable params: 396,554 (1.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

### Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

### Treinamento do modelo

In [ ]:
epochs = 20
batch_size = 64

print("\nIniciando treinamento...")
# Treinando o modelo
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    verbose=1
)


Iniciando treinamento...
Epoch 1/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 68s 80ms/step - accuracy: 0.3217 - loss: 3.9539 - val_accuracy: 0.5976 - val_loss: 1.2036
Epoch 2/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.3906 - loss: 1.6773 - val_accuracy: 0.5979 - val_loss: 1.2042
Epoch 3/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 71s 77ms/step - accuracy: 0.5111 - loss: 1.4105 - val_accuracy: 0.6517 - val_loss: 1.0356
Epoch 4/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5781 - loss: 1.1783 - val_accuracy: 0.6509 - val_loss: 1.0357
Epoch 5/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 61s 78ms/step - accuracy: 0.5641 - loss: 1.2712 - val_accuracy: 0.6778 - val_loss: 0.9797
Epoch 6/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6406 - loss: 1.2108 - val_accuracy: 0.6783 - val_loss: 0.9790
Epoch 7/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 59s 76ms/step - accuracy: 0.5918 - loss: 1.1992 - val_accuracy: 0.6781 - val_loss: 0.9482
Epoch 8/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5781

### Avaliação do modelo

In [ ]:
print("\nAvaliando o modelo...")
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Acurácia nos dados de teste: {test_accuracy:.4f}')
print(f'Loss nos dados de teste: {test_loss:.4f}')



Avaliando o modelo...
Acurácia nos dados de teste: 0.6980
Loss nos dados de teste: 0.8848
